# Module 3 Assignment
### Name: Jaya Marella

In [1]:
#import statements
%matplotlib inline

import pandas as pd
import os
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances
from scipy.spatial.distance import euclidean

In [2]:
#read csv
youtube_df = pd.read_csv('../Module3/data/Youtuber.csv', encoding= 'unicode_escape')
youtube_df

,Rank,Channel Name,Category,Subscribers,Country,Average Views,Average Likes,Average Comments,Content Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,T-Series,Music & Dance,258.4M,India,135.2K,5.6K,223,NaN
2,2.0,MrBeast,Video games,236.1M,United States,104M,4M,74K,Humor
3,3.0,Cocomelon - Nursery Rhymes,Education,171.4M,NaN,5.1M,57.1K,0,NaN
4,4.0,SET India,NaN,167.1M,India,27.9K,996,7,NaN
...,...,...,...,...,...,...,...,...,...
1041,996.0,BBC Earth,NaN,12.5M,United States,80.3K,1.7K,79,NaN
1042,997.0,Jamie Nyland,Animation,12.5M,United States,66.7K,2.5K,104,Humor
1043,998.0,janin al chair,NaN,12.5M,Mexico,1.4M,146.1K,485,NaN
1044,999.0,Myke Towers,Music & Dance,12.5M,Colombia,8.3K,485,8,NaN


In [3]:
youtube_df['Category'].unique()

array([nan, 'Music & Dance', 'Video games', 'Education', 'Animation',
       'Toys', 'Movies', 'News & Politics', 'Daily vlogs', 'Travel',
       'Fitness', 'Beauty', 'Humor', 'Food & Drinks', 'Design/art',
       'Sports', 'Science & Technology', 'Health & Self Help',
       'DIY & Life Hacks', 'ASMR', 'Autos & Vehicles', 'Animals & Pets',
       'Fashion', 'Mystery'], dtype=object)

In [4]:
youtube_df['Country'].unique()

array([nan, 'India', 'United States', 'Brazil', 'Pakistan', 'Russia',
       'Mexico', 'Philippines', 'Indonesia', 'France', 'Thailand',
       'Colombia', 'Iraq', 'Japan', 'Argentina', 'El Salvador', 'Turkey',
       'Saudi Arabia', 'Bangladesh', 'United Kingdom', 'Algeria', 'Spain',
       'Peru', 'South Korea', 'Iran', 'Vietnam', 'Morocco', 'Egypt',
       'Singapore', 'Jordan', 'Italy'], dtype=object)

In [5]:
#change format of column names
old_names = youtube_df.columns
new_names = ['_'.join(c.split()).lower() for c in old_names]
youtube_df.columns = new_names
youtube_df.head()

,rank,channel_name,category,subscribers,country,average_views,average_likes,average_comments,content_type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,T-Series,Music & Dance,258.4M,India,135.2K,5.6K,223,NaN
2,2.0,MrBeast,Video games,236.1M,United States,104M,4M,74K,Humor
3,3.0,Cocomelon - Nursery Rhymes,Education,171.4M,NaN,5.1M,57.1K,0,NaN
4,4.0,SET India,NaN,167.1M,India,27.9K,996,7,NaN


In [6]:
#filter by a specific category and country
youtube_df = youtube_df[youtube_df['category'] == 'Music & Dance'] 
youtube_df = youtube_df[youtube_df['country'] == 'United States']

In [7]:
youtube_df

,rank,channel_name,category,subscribers,country,average_views,average_likes,average_comments,content_type
16,16.0,HYBE LABELS,Music & Dance,73.9M,United States,118.1K,21.6K,215,NaN
33,33.0,Marshmello,Music & Dance,56.9M,United States,88.2K,7.1K,137,NaN
34,34.0,LooLoo Kids - Nursery Rhymes and Children's Songs,Music & Dance,56.1M,United States,237.4K,773,0,NaN
36,36.0,Ariana Grande,Music & Dance,53.5M,United States,310.7K,40.4K,1.5K,NaN
44,44.0,Billie Eilish,Music & Dance,49.1M,United States,363.1K,30.2K,418,NaN
55,55.0,Alan Walker,Music & Dance,44.9M,United States,163.3K,13.1K,469,Movies
102,102.0,Maroon 5,Music & Dance,36.8M,United States,0,0,0,NaN
120,120.0,DisneyMusicVEVO,Music & Dance,34.1M,United States,53.2K,300,15,Movies
127,127.0,NoCopyrightSounds,Music & Dance,33.7M,United States,50.2K,3.5K,86,Animation
140,140.0,SMTOWN,Music & Dance,32.3M,United States,295.3K,26.9K,1.6K,NaN


In [8]:
#fill in null values 
yt_df = youtube_df.dropna(subset=['channel_name']) #from Kaggle
yt_df['country'].fillna('Unknown', inplace=True) #from Kaggle
yt_df['category'].fillna('Unknown', inplace=True)
yt_df['content_type'].fillna('Unknown', inplace=True)

In [9]:
#convert subscribers to numeric values
yt_df['subscribers'] = pd.to_numeric(yt_df['subscribers'].astype(str).str.rstrip('M'), errors='coerce') * 1e6


In [10]:
#change average_views and average_likes to numeric
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

yt_df['average_views'] = yt_df['average_views'].apply(value_to_float)
yt_df['average_likes'] = yt_df['average_likes'].apply(value_to_float)

In [11]:
#set index to channel name
yt_df = yt_df.set_index(['channel_name'])

In [12]:
#remove unnecessary columns 
yt_df.drop(['content_type', 'rank', 'category', 'average_comments', 'country'], axis=1, inplace=True)

In [13]:
#drop duplicates
yt_df = yt_df.drop_duplicates()

In [14]:
yt_df.head(10)

,subscribers,average_views,average_likes
channel_name,,,
HYBE LABELS,73900000.0,118100.0,21600.0
Marshmello,56900000.0,88200.0,7100.0
LooLoo Kids - Nursery Rhymes and Children's Songs,56100000.0,237400.0,0.0
Ariana Grande,53500000.0,310700.0,40400.0
Billie Eilish,49100000.0,363100.0,30200.0
Alan Walker,44900000.0,163300.0,13100.0
Maroon 5,36800000.0,0.0,0.0
DisneyMusicVEVO,34100000.0,53200.0,0.0
NoCopyrightSounds,33700000.0,50200.0,3500.0


In [15]:
#L1 normalization
df_norm = yt_df.divide(yt_df.sum(axis=1), axis=0)
df_norm.head(10)

,subscribers,average_views,average_likes
channel_name,,,
HYBE LABELS,0.998113,0.001595,0.000292
Marshmello,0.998328,0.001547,0.000125
LooLoo Kids - Nursery Rhymes and Children's Songs,0.995786,0.004214,0.000000
Ariana Grande,0.993480,0.005770,0.000750
Billie Eilish,0.992053,0.007336,0.000610
Alan Walker,0.996087,0.003623,0.000291
Maroon 5,1.000000,0.000000,0.000000
DisneyMusicVEVO,0.998442,0.001558,0.000000
NoCopyrightSounds,0.998409,0.001487,0.000104


In [16]:
#check if rows normalized correctly
df_norm.sum(axis=1).head(20)

channel_name
HYBE LABELS                                          1.0
Marshmello                                           1.0
LooLoo Kids - Nursery Rhymes and Children's Songs    1.0
Ariana Grande                                        1.0
Billie Eilish                                        1.0
Alan Walker                                          1.0
Maroon 5                                             1.0
DisneyMusicVEVO                                      1.0
NoCopyrightSounds                                    1.0
SMTOWN                                               1.0
Spinnin' Records                                     1.0
Trap Nation                                          1.0
Ultra Records                                        1.0
JYP Entertainment                                    1.0
Wiz Khalifa                                          1.0
Jason Derulo                                         1.0
America's Got Talent                                 1.0
WORLDSTARHIPHOP   

In [17]:
yt_channel_map = {}

In [18]:
#target channel
target_channel1 = "Olivia Rodrigo"

In [19]:
print(df_norm.loc[target_channel1])

subscribers      0.958660
average_views    0.039121
average_likes    0.002220
Name: Olivia Rodrigo, dtype: float64


In [20]:
#gathering the stats for that channel
target_channel_stats = df_norm.loc[target_channel1]

#generating distances from that channel to all the others
distances = scipy.spatial.distance.cdist(df_norm, [target_channel_stats], metric="euclidean")[:,0]

query_distances = list(zip(yt_df.index, distances))

In [23]:
#sorting distances 
# def get_sortable_element(e):
#     return e[1]
# sorted(query_distances, key=get_sortable_element)

In [24]:
#printing the top ten most similar channels to our target
for similar_channel, similar_channel_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    print('channel name: ', similar_channel, ' ', 'distance: ', similar_channel_score)

channel name:  Olivia Rodrigo   distance:  0.0
channel name:  Crazy Frog   distance:  0.0021150178103815516
channel name:  Lana Del Rey   distance:  0.009638532997118959
channel name:  YoungBoy Never Broke Again   distance:  0.011418635557193912
channel name:  Wild 'N Out   distance:  0.018921613315348636
channel name:  Lyrical Lemonade   distance:  0.024260859583518084
channel name:  JYP Entertainment   distance:  0.036208035669152545
channel name:  Dua Lipa   distance:  0.040012573707479945
channel name:  Tekashi 6ix9ine   distance:  0.04107513364200492
channel name:  Lil Nas X   distance:  0.041728955704534394
